# FiftyOne

## Compute uniqueness

Already included this in the script.

In [1]:
import fiftyone as fo
import fiftyone.brain as fob

existent_datasets = fo.list_datasets()
print(existent_datasets)

['2023.04.07.19.54.02', 'auv', 'merged', 'rov']


In [2]:
# for ds in existent_datasets:
#     dataset = fo.load_dataset(ds)
#     dataset.delete()

# existent_datasets = fo.list_datasets()
# print(existent_datasets)

In [3]:
# dataset2 = fo.load_dataset("2023.04.07.17.29.18")
# dataset2.delete()

## Add predictions

More info here: https://docs.voxel51.com/user_guide/dataset_creation/index.html#model-predictions

In [ ]:
classes = ['Pasiphaea', 'Poeobius meseres', 'Siphonophorae', 'Ctenophora', 'Medusae', 'Eusergestes similis', 'Octopus', 'Larvacean', 'Fish', 'Squid', 'Mysida', 'Worm', 'Echinoderm', 'Other', 'Crustacea', 'Anemone', 'Equipment', 'Coral', 'Sponge', 'Pennatulacea', 'Euphausiacea']

In [ ]:
import fiftyone.utils.yolo as fouy

# And add model predictions
fouy.add_yolo_labels(dataset,
    "predictions",
    "/tmp/yolov4/predictions",
    classes,
)

## List saved views

In [ ]:
dataset.list_saved_views()

In [ ]:
dataset.get_saved_view_info('auv_val')

## Visualise embeddings
Try this out.

## All in one

In [4]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
import pickle

fo.list_datasets()

['2023.04.07.19.54.02', 'auv', 'merged', 'rov']

In [5]:
dataset_name = "2023.04.07.19.54.02"
dataset_dir="/mnt/c/Users/sabri/Documents/github/thesis/datasets/raw"

In [6]:
# dataset = fo.Dataset.from_images_dir(dataset_dir)

# print(f"Dataset created: {dataset_name}.\n")

In [7]:
# classes = ['Pasiphaea',
# 'Poeobius meseres',
# 'Siphonophorae',
# 'Ctenophora',
# 'Medusae',
# 'Eusergestes similis',
# 'Octopus',
# 'Larvacean',
# 'Fish',
# 'Squid',
# 'Mysida',
# 'Worm',
# 'Echinoderm',
# 'Other',
# 'Crustacea',
# 'Anemone',
# 'Equipment',
# 'Coral',
# 'Sponge',
# 'Pennatulacea', 
# 'Euphausiacea']

# # add labels
# import fiftyone.utils.yolo as fouy

# fouy.add_yolo_labels(
#     sample_collection=dataset, 
#     label_field="ground_truth",
#     labels_path="/mnt/c/Users/sabri/Documents/github/thesis/datasets/raw/labels",
#     classes=classes,
#     )

In [8]:
# dataset.persistent = True

In [9]:
dataset = fo.load_dataset(dataset_name)

In [10]:
print(dataset.first())

<Sample: {
    'id': '643058bca3e3e8563050496d',
    'media_type': 'image',
    'filepath': '/mnt/c/Users/sabri/Documents/github/thesis/datasets/raw/images/0001b461-9780-4bd1-8656-425891402580.png',
    'tags': ['rov'],
    'metadata': <ImageMetadata: {
        'size_bytes': 389887,
        'mime_type': 'image/png',
        'width': 720,
        'height': 486,
        'num_channels': 3,
    }>,
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '643058c4a3e3e8563050a836',
                'attributes': {},
                'tags': [],
                'label': 'Pasiphaea',
                'bounding_box': [0.461805, 0.44547300000000006, 0.118056, 0.13786],
                'mask': None,
                'confidence': None,
                'index': None,
            }>,
        ],
    }>,
    'uniqueness': 0.17149335231831642,
    'predictions_auv450': None,
    'predictions_onnx': None,
    'predictions_tflite16': None,
}>


## Tag dataset

In [25]:
import glob
import os
from tqdm import tqdm
import fiftyone as fo

def tag_samples(dataset_name, processed_dir):
    """
    This function tags samples in a given FiftyOne dataset with the tag names extracted from the file names of the provided directory.

    Args:
    dataset_name (str): The name of the FiftyOne dataset to be updated with tags.
    processed_dir (str): The path to the directory containing files with the image file paths. The file names without the extension will be used as tag names.

    Functionality:
    1. Loads the specified dataset using FiftyOne.
    2. Iterates through all the files in the given directory, recursively.
    3. Extracts the tag name from the file name without extension.
    4. Opens the file and iterates through the image file paths.
    5. Matches the image path in the dataset and retrieves the sample.
    6. Appends the tag name to the sample's tag list, if not already present, and saves the sample.
    7. In case of any ValueError, prints an error message with the affected image path.
    """
    dataset = fo.load_dataset(dataset_name)
    for file_path in glob.glob(processed_dir, recursive=True):
        print(f"Getting images in {file_path}")
        tag_name = os.path.splitext(os.path.basename(file_path))[0]

        with open(file_path, 'r') as file:
            for line in tqdm(file):
                image_path = line.strip()
                try:
                    sample = dataset.match({"filepath": image_path}).first()
                    if tag_name not in sample.tags:
                        sample.tags.append(tag_name)
                        sample.save()
                except ValueError as e:
                    print(f"Error: {e} at {image_path}")

dataset_name = "2023.04.07.19.54.02"
processed_dir = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/**/*.txt'

tag_samples(dataset_name, processed_dir)

Getting images in /mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/auv/auv_train.txt


450it [00:01, 247.96it/s]


Getting images in /mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/auv/auv_val.txt


79it [00:00, 280.46it/s]


Getting images in /mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/auv_increasing_random/train100.txt


...

4747it [00:06, 773.30it/s]


Getting images in /mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/unique_least/least_unique.txt


50it [00:00, 167.87it/s]


Getting images in /mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/unique_most/most_unique.txt


50it [00:00, 147.61it/s]


In [10]:
fo.pprint(dataset.stats(include_media=True))

{
    'samples_count': 24265,
    'samples_bytes': 17069973,
    'samples_size': '16.3MB',
    'media_bytes': 40973704444,
    'media_size': '38.2GB',
    'total_bytes': 40990774417,
    'total_size': '38.2GB',
}


In [ ]:
classes = ['Pasiphaea',
'Poeobius meseres',
'Siphonophorae',
'Ctenophora',
'Medusae',
'Eusergestes similis',
'Octopus',
'Larvacean',
'Fish',
'Squid',
'Mysida',
'Worm',
'Echinoderm',
'Other',
'Crustacea',
'Anemone',
'Equipment',
'Coral',
'Sponge',
'Pennatulacea', 
'Euphausiacea']

# add labels
import fiftyone.utils.yolo as fouy

fouy.add_yolo_labels(
    sample_collection=dataset, 
    label_field="predictions_tflite16",
    labels_path="/mnt/c/Users/sabri/Documents/github/thesis/artifacts/predictions/predict_tflite16/labels",
    classes=classes,
    )

In [ ]:
# Launch App instance
session = fo.launch_app(dataset)

In [ ]:
session.close()

### Embeddings
How to do this explained here: https://docs.voxel51.com/tutorials/image_embeddings.html

In [ ]:
# Open the file in binary mode
with open('embeddings.pkl', 'rb') as file:
    # Call load method to deserialze
    embeddings = pickle.load(file)

#### Compute embeddings
I have already done this and pickled it.

In [ ]:
# import fiftyone.zoo as foz

# # Compute embeddings
# # You will likely want to run this on a machine with GPU, as this requires
# # running inference on 10,000 images
# model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
# embeddings = dataset.compute_embeddings(model)

# # Open a file and use dump()
# with open('embeddings.pkl', 'wb') as file:
#     # A new file will be created
#     pickle.dump(embeddings, file)

In [ ]:
# Compute visualization
results = fob.compute_visualization(
    dataset, embeddings=embeddings, seed=42, brain_key="img_viz"
)

In [ ]:
# Object patch embeddings
fob.compute_visualization(
    dataset, patches_field="ground_truth", brain_key="gt_viz"
)

In [ ]:
print(type(results))
print(results.points.shape)

In [ ]:
fob.compute_uniqueness(dataset, embeddings=embeddings)

In [ ]:
fob.compute_similarity(dataset, embeddings=embeddings)

In [ ]:
dataset.save()

Computing patch embeddings is breaking the kernel.

In [ ]:
# %%capture
# dataset.compute_patch_embeddings(
#     model, 
#     "ground_truth", 
#     embeddings_field = "gt_embed"
# )

In [ ]:
dataset.list_brain_runs()

In [ ]:
dataset.count_values("ground_truth.detections.label")

In [ ]:
results = dataset.load_brain_results("img_viz")

In [ ]:
# Plot embeddings colored by ground truth label
plot = results.visualize(labels="ground_truth.detections.label")
plot.show(height=520)

# # Attach plot to session
# session.plots.attach(plot)

In [ ]:
# Launch App instance
session = fo.launch_app(dataset)

In [ ]:
from fiftyone import ViewField as F

auv_view = dataset.match(F("filepath").contains_str("output"))
auv_view.tag_samples("auv")